In [ ]:
%sql -- Extracao completa de ordens com campos enriquecidos

WITH ordens_filtradas AS (
    SELECT aufk.DI_SEQUENCE_NUMBER, aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA', 'GAA', 'GRA')
      AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
      AND aufk.DI_SEQUENCE_NUMBER = (
          SELECT MAX(a.DI_SEQUENCE_NUMBER)
          FROM poseidon_uc.saps4hanarawview.aufk AS a
          WHERE a.MANDT = aufk.MANDT
            AND a.AUFNR = aufk.AUFNR
      )
      AND aufk.DI_OPERATION_TYPE != 'D'
),
aufk_atual AS (
    SELECT aufk.*
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    INNER JOIN ordens_filtradas of
        ON of.AUFNR = aufk.AUFNR
       AND of.MANDT = aufk.MANDT
       AND of.DI_SEQUENCE_NUMBER = aufk.DI_SEQUENCE_NUMBER
),
afih_filtrado AS (
    SELECT afih.*
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT
          AND a.AUFNR = afih.AUFNR
    )
),
afko_filtrado AS (
    SELECT afko.*
    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT
          AND a.AUFNR = afko.AUFNR
    )
),
afvc_filtrado AS (
    SELECT afvc.*
    FROM poseidon_uc.saps4hanarawview.afvc AS afvc
    WHERE afvc.VORNR IN ('0010', '010', '10')
      AND afvc.DI_SEQUENCE_NUMBER = (
          SELECT MAX(a.DI_SEQUENCE_NUMBER)
          FROM poseidon_uc.saps4hanarawview.afvc AS a
          WHERE a.MANDT = afvc.MANDT
            AND a.AUFPL = afvc.AUFPL
            AND a.VORNR = afvc.VORNR
      )
),
operacoes AS (
    SELECT afko.MANDT, afko.AUFNR,
           CONCAT_WS(' ', COLLECT_LIST(afvc.LTXA1)) AS texto_breve_operacao
    FROM afko_filtrado AS afko
    LEFT JOIN afvc_filtrado AS afvc
        ON afvc.MANDT = afko.MANDT
       AND afvc.AUFPL = afko.AUFPL
    GROUP BY afko.MANDT, afko.AUFNR
),
iloa_filtrado AS (
    SELECT iloa.*
    FROM poseidon_uc.saps4hanarawview.iloa AS iloa
    WHERE iloa.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.iloa AS a
        WHERE a.MANDT = iloa.MANDT
          AND a.ILOAN = iloa.ILOAN
    )
),
status_usuario AS (
    SELECT aufk.AUFNR,
        CONCAT_WS(' ', COLLECT_LIST(tj30t.TXT04)) AS status_usuario
    FROM aufk_atual AS aufk
    LEFT JOIN poseidon_uc.saps4hanarawview.jest AS jest
        ON jest.MANDT = aufk.MANDT
        AND jest.OBJNR = aufk.OBJNR
        AND jest.STAT LIKE 'E%'
        AND jest.INACT IS NULL
    LEFT JOIN poseidon_uc.saps4hanarawview.jsto AS jsto
        ON jsto.OBJNR = aufk.OBJNR
    LEFT JOIN poseidon_uc.saps4hanarawview.tj30t AS tj30t
        ON tj30t.ESTAT = jest.STAT
        AND tj30t.SPRAS = 'P'
        AND tj30t.STSMA = jsto.STSMA
    GROUP BY aufk.AUFNR
)

SELECT
    aufk.AUFNR AS ordem,
    aufk.KTEXT AS texto_breve,
    aufk.AUART AS tipo_ordem,
    aufk.BUKRS AS empresa,
    aufk.VAPLZ AS centro_de_trabalho_principal,
    aufk.WAWRK AS centro,
    aufk.STORT AS grupo_planejamento,
    su.status_usuario AS status_usuario,
    iloa.TPLNR AS local_de_instalacao,
    afih.WARPL AS plano_de_manutencao,
    afko.GSTRP AS data_base_inicio,
    afko.GLTRP AS data_base_fim,
    afko.GSTRI AS data_inicio_real,
    afko.GETRI AS data_fim_real,
    afih.ILART AS tipo_atividade_manutencao,
    afih.QMNUM AS nota,
    t356_t.PRIOKX AS texto_prioridade,
    afih.EQUNR AS equipamento,
    afih.ADPSP AS elem_ref_PM_PS,
    afih.ADDAT AS data_referencia,
    aufk.ERDAT AS data_entrada,
    afih.WAPOS AS item_manutencao,
    op.texto_breve_operacao AS texto_breve_operacao

FROM aufk_atual AS aufk
LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT
   AND afih.AUFNR = aufk.AUFNR
LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT
   AND afko.AUFNR = aufk.AUFNR
LEFT JOIN operacoes AS op
    ON op.MANDT = aufk.MANDT
   AND op.AUFNR = aufk.AUFNR
LEFT JOIN status_usuario AS su
    ON su.AUFNR = aufk.AUFNR
LEFT JOIN iloa_filtrado AS iloa
    ON iloa.MANDT = afih.MANDT
   AND iloa.ILOAN = afih.ILOAN
LEFT JOIN poseidon_uc.saps4hanarawview.t356_t AS t356_t
    ON t356_t.PRIOK = afih.PRIOK
   AND t356_t.ARTPR = afih.ARTPR
   AND t356_t.SPRAS = 'P'

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração completa de ordens 

WITH ordens_filtradas AS (
    SELECT 
        aufk.AUFNR, 
        aufk.MANDT,
        aufk.KTEXT,
        aufk.AUART,
        aufk.BUKRS,
        aufk.VAPLZ,
        aufk.WAWRK,
        aufk.STORT,
        aufk.ERDAT,
        aufk.OBJNR
        
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    -- LIMIT 10  -- Processar em lotes
),
afih_filtrado AS (
    SELECT 
        afih.MANDT, 
        afih.AUFNR,
        afih.ILOAN, 
        afih.WARPL,
        afih.ILART,
        afih.QMNUM,
        afih.EQUNR,
        afih.ADPSP,
        afih.ADDAT,
        afih.WAPOS,
        afih.PRIOK,
        afih.ARTPR

    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
),
afko_filtrado AS (
    SELECT 
        afko.MANDT,
        afko.AUFNR,
        afko.AUFPL,
        afko.GSTRP,
        afko.GLTRP,
        afko.GSTRI,
        afko.GETRI

    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
),
afvc_filtrado AS (
    SELECT 
        afvc.MANDT,
        afvc.AUFPL,
        afvc.LTXA1

    FROM poseidon_uc.saps4hanarawview.afvc AS afvc
    WHERE afvc.VORNR IN ('0010', '010', '10')
    AND afvc.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afvc AS a
        WHERE a.MANDT = afvc.MANDT
        AND a.AUFPL = afvc.AUFPL
        AND a.VORNR IN ('0010', '010', '10')
    )
)

SELECT
    aufk.AUFNR      AS ordem,
    aufk.KTEXT      AS texto_breve,
    aufk.AUART      AS tipo_ordem,
    aufk.BUKRS      AS empresa,
    aufk.VAPLZ      AS centro_de_trabalho_principal,
    aufk.WAWRK      AS centro,
    aufk.STORT      AS grupo_planejamento,
    CONCAT_WS(' ', COLLECT_LIST(tj30t.TXT04))   AS status_usuario,
    iloa.TPLNR      AS local_de_instalacao,
    afih.WARPL      AS plano_de_manutencao,
    afko.GSTRP      AS data_base_inicio,
    afko.GLTRP      AS data_base_fim,
    afko.GSTRI      AS data_inicio_real,
    afko.GETRI      AS data_fim_real,
    afih.ILART      AS tipo_atividade_manutencao,
    afih.QMNUM      AS nota,
    t356_t.PRIOKX   AS texto_prioridade,
    afih.EQUNR      AS equipamento,
    afih.ADPSP      AS elem_ref_PM_PS,
    afih.ADDAT      AS data_referencia,
    aufk.ERDAT      AS data_entrada,
    afih.WAPOS      AS item_manutencao,
    afvc.LTXA1      AS texto_breve_operacao

FROM ordens_filtradas AS aufk

LEFT JOIN poseidon_uc.saps4hanarawview.jest AS jest
    ON jest.MANDT = aufk.MANDT 
    AND jest.OBJNR = aufk.OBJNR
    AND jest.STAT LIKE 'E%'
    AND jest.INACT IS NULL

LEFT JOIN poseidon_uc.saps4hanarawview.jsto AS jsto
    ON jsto.OBJNR = aufk.OBJNR

LEFT JOIN poseidon_uc.saps4hanarawview.tj30t AS tj30t
    ON jest.STAT = tj30t.ESTAT
    AND tj30t.SPRAS = 'P'
    AND tj30t.STSMA = jsto.STSMA

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.MANDT = afih.MANDT AND iloa.ILOAN = afih.ILOAN

LEFT JOIN poseidon_uc.saps4hanarawview.t356_t AS t356_t
    ON t356_t.PRIOK = afih.PRIOK
    AND t356_t.ARTPR = afih.ARTPR
    AND t356_t.SPRAS = 'P'

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afvc_filtrado AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

GROUP BY aufk.AUFNR

In [ ]:
%sql -- Extração completa de ordens 

WITH ordens_filtradas AS (
    SELECT 
        aufk.AUFNR, 
        aufk.MANDT,
        aufk.KTEXT,
        aufk.AUART,
        aufk.BUKRS,
        aufk.VAPLZ,
        aufk.WAWRK,
        aufk.STORT,
        aufk.ERDAT,
        aufk.OBJNR,
        aufk.DI_OPERATION_TYPE,
        aufk.DI_SEQUENCE_NUMBER
        
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    -- LIMIT 10  -- Processar em lotes
),
afih_filtrado AS (
    SELECT 
        afih.MANDT, 
        afih.AUFNR,
        afih.ILOAN, 
        afih.WARPL,
        afih.ILART,
        afih.QMNUM,
        afih.EQUNR,
        afih.ADPSP,
        afih.ADDAT,
        afih.WAPOS,
        afih.DI_OPERATION_TYPE,
        afih.DI_SEQUENCE_NUMBER,
        t356_t.PRIOKX AS prioridade,
        t356_t.DI_OPERATION_TYPE AS t356_t_DI_OPERATION_TYPE,
        t356_t.DI_SEQUENCE_NUMBER AS t356_t_DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afih AS afih

    LEFT JOIN poseidon_uc.saps4hanarawview.t356_t AS t356_t
        ON t356_t.PRIOK = afih.PRIOK
        AND t356_t.ARTPR = afih.ARTPR
        AND t356_t.SPRAS = 'P'

    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
),
afko_filtrado AS (
    SELECT 
        afko.MANDT,
        afko.AUFNR,
        afko.AUFPL,
        afko.GSTRP,
        afko.GLTRP,
        afko.GSTRI,
        afko.GETRI,
        afko.DI_OPERATION_TYPE,
        afko.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
),
afvc_filtrado AS (
    SELECT 
        afvc.MANDT,
        afvc.AUFPL,
        afvc.LTXA1,
        afvc.DI_OPERATION_TYPE,
        afvc.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afvc AS afvc
    WHERE afvc.VORNR IN ('0010', '010', '10')
    AND afvc.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afvc AS a
        WHERE a.MANDT = afvc.MANDT
        AND a.AUFPL = afvc.AUFPL
        AND a.VORNR IN ('0010', '010', '10')
    )
),
status_usuario AS (
    SELECT 
        aufk.AUFNR,
        CONCAT_WS(' ', COLLECT_LIST(tj30t.TXT04)) AS status_usuario
        -- ,
        -- jest.DI_OPERATION_TYPE AS jest_DI_OPERATION_TYPE,
        -- jsto.DI_OPERATION_TYPE AS jsto_DI_OPERATION_TYPE,
        -- tj30t.DI_OPERATION_TYPE AS tj30t_DI_OPERATION_TYPE

    FROM ordens_filtradas AS aufk
    
    LEFT JOIN poseidon_uc.saps4hanarawview.jest AS jest
        ON jest.MANDT = aufk.MANDT
        AND jest.OBJNR = aufk.OBJNR
        AND jest.STAT LIKE 'E%'
        AND jest.INACT IS NULL

    LEFT JOIN poseidon_uc.saps4hanarawview.jsto AS jsto
        ON jsto.OBJNR = aufk.OBJNR

    LEFT JOIN poseidon_uc.saps4hanarawview.tj30t AS tj30t
        ON tj30t.ESTAT = jest.STAT
        AND tj30t.SPRAS = 'P'
        AND tj30t.STSMA = jsto.STSMA

    GROUP BY aufk.AUFNR
),
iloa_filtrado AS (
    SELECT DISTINCT
        iloa.DI_SEQUENCE_NUMBER,
        iloa.MANDT,
        iloa.ILOAN,
        iloa.TPLNR,
        iloa.DI_OPERATION_TYPE

    FROM poseidon_uc.saps4hanarawview.iloa AS iloa
    WHERE iloa.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.iloa AS a
        WHERE a.MANDT = iloa.MANDT 
        AND a.ILOAN = iloa.ILOAN
    )
)

SELECT
    aufk.AUFNR          AS ordem,
    aufk.KTEXT          AS texto_breve,
    aufk.AUART          AS tipo_ordem,
    aufk.BUKRS          AS empresa,
    aufk.VAPLZ          AS centro_de_trabalho_principal,
    aufk.WAWRK          AS centro,
    aufk.STORT          AS grupo_planejamento,
    su.status_usuario   AS status_usuario,
    iloa.TPLNR          AS local_de_instalacao,

    regexp_replace(afih.WARPL, '^0+', '')          
                        AS plano_de_manutencao,
    afko.GSTRP          AS data_base_inicio,
    afko.GLTRP          AS data_base_fim,
    afko.GSTRI          AS data_inicio_real,
    afko.GETRI          AS data_fim_real,
    afih.ILART          AS tipo_atividade_manutencao,

    regexp_replace(afih.QMNUM, '^0+', '')          
                        AS nota,
    afih.prioridade     AS texto_prioridade,
    afih.EQUNR          AS equipamento,
    afih.ADPSP          AS elem_ref_PM_PS,
    afih.ADDAT          AS data_referencia,
    aufk.ERDAT          AS data_entrada,

    regexp_replace(afih.WAPOS, '^0+', '')          
                        AS item_manutencao,
    afvc.LTXA1          AS texto_breve_operacao,

    aufk.DI_OPERATION_TYPE          AS aufk_DI_OPERATION_TYPE,
    aufk.DI_SEQUENCE_NUMBER         AS aufk_DI_SEQUENCE_NUMBER,

    afih.DI_OPERATION_TYPE          AS afih_DI_OPERATION_TYPE,
    afih.DI_SEQUENCE_NUMBER         AS afih_DI_SEQUENCE_NUMBER,

    afih.t356_t_DI_OPERATION_TYPE   AS t356_t_DI_OPERATION_TYPE,
    afih.t356_t_DI_SEQUENCE_NUMBER  AS t356_t_DI_SEQUENCE_NUMBER,

    iloa.DI_OPERATION_TYPE          AS iloa_DI_OPERATION_TYPE,
    iloa.DI_SEQUENCE_NUMBER         AS iloa_DI_SEQUENCE_NUMBER,

    afko.DI_OPERATION_TYPE          AS afko_DI_OPERATION_TYPE,
    afko.DI_SEQUENCE_NUMBER         AS afko_DI_SEQUENCE_NUMBER,

    afvc.DI_OPERATION_TYPE          AS afvc_DI_OPERATION_TYPE,
    afvc.DI_SEQUENCE_NUMBER         AS afvc_DI_SEQUENCE_NUMBER

FROM ordens_filtradas AS aufk

LEFT JOIN status_usuario AS su
    ON su.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN iloa_filtrado AS iloa
    ON iloa.MANDT = afih.MANDT AND iloa.ILOAN = afih.ILOAN

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afvc_filtrado AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração completa de ordens
-- com filtro por maior numero de sequencia e tipo de operaçao válida

WITH ordens_filtradas AS (
    SELECT 
        aufk.AUFNR, 
        aufk.MANDT,
        aufk.KTEXT,
        aufk.AUART,
        aufk.BUKRS,
        aufk.VAPLZ,
        aufk.WAWRK,
        aufk.STORT,
        aufk.ERDAT,
        aufk.OBJNR,
        aufk.DI_OPERATION_TYPE,
        aufk.DI_SEQUENCE_NUMBER
        
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    AND aufk.DI_OPERATION_TYPE != 'D'
    -- LIMIT 10  -- Processar em lotes
),
t356_t_filtrado AS (
    SELECT 
        t356_t.PRIOK,
        t356_t.ARTPR,
        t356_t.SPRAS,
        t356_t.PRIOKX,
        t356_t.DI_OPERATION_TYPE,
        t356_t.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.t356_t AS t356_t
    WHERE t356_t.SPRAS = 'P'
    AND t356_t.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.t356_t AS a
        WHERE a.PRIOK = t356_t.PRIOK
        AND a.ARTPR = t356_t.ARTPR
        AND a.SPRAS = 'P'
    )
    AND t356_t.DI_OPERATION_TYPE != 'D'
),
afih_filtrado AS (
    SELECT 
        afih.MANDT, 
        afih.AUFNR,
        afih.ILOAN, 
        afih.WARPL,
        afih.ILART,
        afih.QMNUM,
        afih.EQUNR,
        afih.ADPSP,
        afih.ADDAT,
        afih.WAPOS,
        afih.DI_OPERATION_TYPE,
        afih.DI_SEQUENCE_NUMBER,
        t356_t.PRIOKX AS prioridade,
        t356_t.DI_OPERATION_TYPE AS t356_t_DI_OPERATION_TYPE,
        t356_t.DI_SEQUENCE_NUMBER AS t356_t_DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afih AS afih

    LEFT JOIN t356_t_filtrado AS t356_t
        ON t356_t.PRIOK = afih.PRIOK
        AND t356_t.ARTPR = afih.ARTPR
        AND t356_t.SPRAS = 'P'

    WHERE afih.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afih AS a
        WHERE a.MANDT = afih.MANDT 
        AND a.AUFNR = afih.AUFNR
    )
    AND afih.DI_OPERATION_TYPE != 'D'
),
afko_filtrado AS (
    SELECT 
        afko.MANDT,
        afko.AUFNR,
        afko.AUFPL,
        afko.GSTRP,
        afko.GLTRP,
        afko.GSTRI,
        afko.GETRI,
        afko.DI_OPERATION_TYPE,
        afko.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afko AS a
        WHERE a.MANDT = afko.MANDT 
        AND a.AUFNR = afko.AUFNR
    )
    AND afko.DI_OPERATION_TYPE != 'D'
),
afvc_filtrado AS (
    SELECT 
        afvc.MANDT,
        afvc.AUFPL,
        afvc.LTXA1,
        afvc.DI_OPERATION_TYPE,
        afvc.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afvc AS afvc
    WHERE afvc.VORNR IN ('0010', '010', '10')
    AND afvc.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.afvc AS a
        WHERE a.MANDT = afvc.MANDT
        AND a.AUFPL = afvc.AUFPL
        AND a.VORNR IN ('0010', '010', '10')
    )
    AND afvc.DI_OPERATION_TYPE != 'D'
),
jest_filtrado AS (
    SELECT 
        jest.MANDT,
        jest.OBJNR,
        jest.STAT,
        jest.INACT,
        jest.DI_OPERATION_TYPE,
        jest.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jest AS jest
    WHERE jest.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jest AS a
        WHERE a.MANDT = jest.MANDT
        AND a.OBJNR = jest.OBJNR
        AND a.STAT = jest.STAT
    )
    AND jest.DI_OPERATION_TYPE != 'D'
),
jsto_filtrado AS (
    SELECT 
        jsto.OBJNR,
        jsto.STSMA,
        jsto.DI_OPERATION_TYPE,
        jsto.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jsto AS jsto
    WHERE jsto.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.jsto AS a
        WHERE a.OBJNR = jsto.OBJNR
    )
    AND jsto.DI_OPERATION_TYPE != 'D'
),
tj30_filtrado AS (
    SELECT 
        tj30.STSMA,
        tj30.ESTAT,
        tj30.LINEP

    FROM poseidon_uc.saps4hanarawview.tj30 AS tj30
    WHERE tj30.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30 AS a
        WHERE a.ESTAT = tj30.ESTAT
        AND a.STSMA = tj30.STSMA
    )
    AND tj30.DI_OPERATION_TYPE != 'D'
),
tj30t_filtrado AS (
    SELECT 
        tj30t.ESTAT,
        tj30t.SPRAS,
        tj30t.STSMA,
        tj30t.TXT04,
        tj30t.DI_OPERATION_TYPE,
        tj30t.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.tj30t AS tj30t
    WHERE tj30t.SPRAS = 'P'
    AND tj30t.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.tj30t AS a
        WHERE a.ESTAT = tj30t.ESTAT
        AND a.SPRAS = 'P'
        AND a.STSMA = tj30t.STSMA
    )
    AND tj30t.DI_OPERATION_TYPE != 'D'
),
status_usuario AS (
    SELECT 
        aufk.AUFNR,
        -- CONCAT_WS(' ', COLLECT_LIST(tj30t.TXT04 ORDER BY tj30.LINEP)) AS status_usuario
        CONCAT_WS(' ', 
            transform(
                array_sort(collect_list(struct(tj30.LINEP, tj30t.TXT04))),
                x -> x.TXT04
            )
        ) AS status_usuario
        -- ,
        -- jest.DI_OPERATION_TYPE AS jest_DI_OPERATION_TYPE,
        -- jsto.DI_OPERATION_TYPE AS jsto_DI_OPERATION_TYPE,
        -- tj30t.DI_OPERATION_TYPE AS tj30t_DI_OPERATION_TYPE

    FROM ordens_filtradas AS aufk
    
    LEFT JOIN jest_filtrado AS jest
        ON jest.MANDT = aufk.MANDT
        AND jest.OBJNR = aufk.OBJNR
        AND jest.STAT LIKE 'E%'
        AND jest.INACT IS NULL

    LEFT JOIN jsto_filtrado AS jsto
        ON jsto.OBJNR = aufk.OBJNR

    LEFT JOIN tj30_filtrado AS tj30
        ON jsto.STSMA = tj30.STSMA AND jest.STAT = tj30.ESTAT

    LEFT JOIN tj30t_filtrado AS tj30t
        ON tj30t.ESTAT = jest.STAT
        AND tj30t.SPRAS = 'P'
        AND tj30t.STSMA = jsto.STSMA

    GROUP BY aufk.AUFNR

),
iloa_filtrado AS (
    SELECT DISTINCT
        iloa.DI_SEQUENCE_NUMBER,
        iloa.MANDT,
        iloa.ILOAN,
        iloa.TPLNR,
        iloa.DI_OPERATION_TYPE

    FROM poseidon_uc.saps4hanarawview.iloa AS iloa
    WHERE iloa.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.iloa AS a
        WHERE a.MANDT = iloa.MANDT 
        AND a.ILOAN = iloa.ILOAN
    )
    AND iloa.DI_OPERATION_TYPE != 'D'
)

SELECT
    aufk.AUFNR          AS ordem,
    aufk.KTEXT          AS texto_breve,
    aufk.AUART          AS tipo_ordem,
    aufk.BUKRS          AS empresa,
    aufk.VAPLZ          AS centro_de_trabalho_principal,
    aufk.WAWRK          AS centro,
    aufk.STORT          AS grupo_planejamento,
    su.status_usuario   AS status_usuario,
    iloa.TPLNR          AS local_de_instalacao,

    regexp_replace(afih.WARPL, '^0+', '')          
                        AS plano_de_manutencao,
    afko.GSTRP          AS data_base_inicio,
    afko.GLTRP          AS data_base_fim,
    afko.GSTRI          AS data_inicio_real,
    afko.GETRI          AS data_fim_real,
    afih.ILART          AS tipo_atividade_manutencao,

    regexp_replace(afih.QMNUM, '^0+', '')          
                        AS nota,
    afih.prioridade     AS texto_prioridade,
    afih.EQUNR          AS equipamento,
    afih.ADPSP          AS elem_ref_PM_PS,
    afih.ADDAT          AS data_referencia,
    aufk.ERDAT          AS data_entrada,

    regexp_replace(afih.WAPOS, '^0+', '')          
                        AS item_manutencao,
    afvc.LTXA1          AS texto_breve_operacao,

    aufk.DI_OPERATION_TYPE          AS aufk_DI_OPERATION_TYPE,
    aufk.DI_SEQUENCE_NUMBER         AS aufk_DI_SEQUENCE_NUMBER,

    afih.DI_OPERATION_TYPE          AS afih_DI_OPERATION_TYPE,
    afih.DI_SEQUENCE_NUMBER         AS afih_DI_SEQUENCE_NUMBER,

    afih.t356_t_DI_OPERATION_TYPE   AS t356_t_DI_OPERATION_TYPE,
    afih.t356_t_DI_SEQUENCE_NUMBER  AS t356_t_DI_SEQUENCE_NUMBER,

    iloa.DI_OPERATION_TYPE          AS iloa_DI_OPERATION_TYPE,
    iloa.DI_SEQUENCE_NUMBER         AS iloa_DI_SEQUENCE_NUMBER,

    afko.DI_OPERATION_TYPE          AS afko_DI_OPERATION_TYPE,
    afko.DI_SEQUENCE_NUMBER         AS afko_DI_SEQUENCE_NUMBER,

    afvc.DI_OPERATION_TYPE          AS afvc_DI_OPERATION_TYPE,
    afvc.DI_SEQUENCE_NUMBER         AS afvc_DI_SEQUENCE_NUMBER

FROM ordens_filtradas AS aufk

LEFT JOIN status_usuario AS su
    ON su.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN iloa_filtrado AS iloa
    ON iloa.MANDT = afih.MANDT AND iloa.ILOAN = afih.ILOAN

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afvc_filtrado AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

ORDER BY aufk.AUFNR

In [ ]:
%sql -- Extração completa de ordens (OTIMIZADO)
-- Usando ROW_NUMBER + QUALIFY em vez de MAX correlacionado
-- Broadcast hints para tabelas pequenas de domínio/texto

WITH ordens_filtradas AS (
    SELECT 
        aufk.AUFNR, 
        aufk.MANDT,
        aufk.KTEXT,
        aufk.AUART,
        aufk.BUKRS,
        aufk.VAPLZ,
        aufk.WAWRK,
        aufk.STORT,
        aufk.ERDAT,
        aufk.OBJNR,
        aufk.DI_OPERATION_TYPE,
        aufk.DI_SEQUENCE_NUMBER
        
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
      AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
      AND aufk.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY aufk.AUFNR ORDER BY aufk.DI_SEQUENCE_NUMBER DESC) = 1
),
t356_t_filtrado AS (
    SELECT 
        t356_t.PRIOK,
        t356_t.ARTPR,
        t356_t.SPRAS,
        t356_t.PRIOKX,
        t356_t.DI_OPERATION_TYPE,
        t356_t.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.t356_t AS t356_t
    WHERE t356_t.SPRAS = 'P'
      AND t356_t.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY t356_t.PRIOK, t356_t.ARTPR ORDER BY t356_t.DI_SEQUENCE_NUMBER DESC) = 1
),
afih_base AS (
    SELECT 
        afih.MANDT, 
        afih.AUFNR,
        afih.ILOAN, 
        afih.WARPL,
        afih.ILART,
        afih.QMNUM,
        afih.EQUNR,
        afih.ADPSP,
        afih.ADDAT,
        afih.WAPOS,
        afih.PRIOK,
        afih.ARTPR,
        afih.DI_OPERATION_TYPE,
        afih.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY afih.MANDT, afih.AUFNR ORDER BY afih.DI_SEQUENCE_NUMBER DESC) = 1
),
afih_filtrado AS (
    SELECT 
        afih.MANDT, 
        afih.AUFNR,
        afih.ILOAN, 
        afih.WARPL,
        afih.ILART,
        afih.QMNUM,
        afih.EQUNR,
        afih.ADPSP,
        afih.ADDAT,
        afih.WAPOS,
        afih.DI_OPERATION_TYPE,
        afih.DI_SEQUENCE_NUMBER,
        t356_t.PRIOKX AS prioridade,
        t356_t.DI_OPERATION_TYPE AS t356_t_DI_OPERATION_TYPE,
        t356_t.DI_SEQUENCE_NUMBER AS t356_t_DI_SEQUENCE_NUMBER

    FROM afih_base AS afih
    LEFT JOIN t356_t_filtrado AS t356_t
        ON t356_t.PRIOK = afih.PRIOK
        AND t356_t.ARTPR = afih.ARTPR
),
afko_filtrado AS (
    SELECT 
        afko.MANDT,
        afko.AUFNR,
        afko.AUFPL,
        afko.GSTRP,
        afko.GLTRP,
        afko.GSTRI,
        afko.GETRI,
        afko.DI_OPERATION_TYPE,
        afko.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afko AS afko
    WHERE afko.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY afko.MANDT, afko.AUFNR ORDER BY afko.DI_SEQUENCE_NUMBER DESC) = 1
),
afvc_filtrado AS (
    SELECT 
        afvc.MANDT,
        afvc.AUFPL,
        afvc.LTXA1,
        afvc.DI_OPERATION_TYPE,
        afvc.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.afvc AS afvc
    WHERE afvc.VORNR IN ('0010', '010', '10')
      AND afvc.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY afvc.MANDT, afvc.AUFPL, afvc.VORNR ORDER BY afvc.DI_SEQUENCE_NUMBER DESC) = 1
),
jest_filtrado AS (
    SELECT 
        jest.MANDT,
        jest.OBJNR,
        jest.STAT,
        jest.INACT,
        jest.DI_OPERATION_TYPE,
        jest.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jest AS jest
    WHERE jest.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY jest.MANDT, jest.OBJNR, jest.STAT ORDER BY jest.DI_SEQUENCE_NUMBER DESC) = 1
),
jsto_filtrado AS (
    SELECT 
        jsto.OBJNR,
        jsto.STSMA,
        jsto.DI_OPERATION_TYPE,
        jsto.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.jsto AS jsto
    WHERE jsto.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY jsto.OBJNR ORDER BY jsto.DI_SEQUENCE_NUMBER DESC) = 1
),
tj30_filtrado AS (
    SELECT 
        tj30.STSMA,
        tj30.ESTAT,
        tj30.LINEP

    FROM poseidon_uc.saps4hanarawview.tj30 AS tj30
    WHERE tj30.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY tj30.ESTAT, tj30.STSMA ORDER BY tj30.DI_SEQUENCE_NUMBER DESC) = 1
),
tj30t_filtrado AS (
    SELECT 
        tj30t.ESTAT,
        tj30t.SPRAS,
        tj30t.STSMA,
        tj30t.TXT04,
        tj30t.DI_OPERATION_TYPE,
        tj30t.DI_SEQUENCE_NUMBER

    FROM poseidon_uc.saps4hanarawview.tj30t AS tj30t
    WHERE tj30t.SPRAS = 'P'
      AND tj30t.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY tj30t.ESTAT, tj30t.STSMA ORDER BY tj30t.DI_SEQUENCE_NUMBER DESC) = 1
),
status_usuario AS (
    SELECT 
        aufk.AUFNR,
        CONCAT_WS(' ', 
            transform(
                array_sort(collect_list(struct(tj30.LINEP, tj30t.TXT04))),
                x -> x.TXT04
            )
        ) AS status_usuario

    FROM ordens_filtradas AS aufk
    
    LEFT JOIN jest_filtrado AS jest
        ON jest.MANDT = aufk.MANDT
        AND jest.OBJNR = aufk.OBJNR
        AND jest.STAT LIKE 'E%'
        AND jest.INACT IS NULL

    LEFT JOIN jsto_filtrado AS jsto
        ON jsto.OBJNR = aufk.OBJNR

    LEFT JOIN tj30_filtrado AS tj30
        ON jsto.STSMA = tj30.STSMA AND jest.STAT = tj30.ESTAT

    LEFT JOIN tj30t_filtrado AS tj30t
        ON tj30t.ESTAT = jest.STAT
        AND tj30t.STSMA = jsto.STSMA

    GROUP BY aufk.AUFNR
),
iloa_filtrado AS (
    SELECT 
        iloa.DI_SEQUENCE_NUMBER,
        iloa.MANDT,
        iloa.ILOAN,
        iloa.TPLNR,
        iloa.DI_OPERATION_TYPE

    FROM poseidon_uc.saps4hanarawview.iloa AS iloa
    WHERE iloa.DI_OPERATION_TYPE != 'D'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY iloa.MANDT, iloa.ILOAN ORDER BY iloa.DI_SEQUENCE_NUMBER DESC) = 1
)

SELECT
    aufk.AUFNR          AS ordem,
    aufk.KTEXT          AS texto_breve,
    aufk.AUART          AS tipo_ordem,
    aufk.BUKRS          AS empresa,
    aufk.VAPLZ          AS centro_de_trabalho_principal,
    aufk.WAWRK          AS centro,
    aufk.STORT          AS grupo_planejamento,
    su.status_usuario   AS status_usuario,
    iloa.TPLNR          AS local_de_instalacao,

    regexp_replace(afih.WARPL, '^0+', '')          
                        AS plano_de_manutencao,
    afko.GSTRP          AS data_base_inicio,
    afko.GLTRP          AS data_base_fim,
    afko.GSTRI          AS data_inicio_real,
    afko.GETRI          AS data_fim_real,
    afih.ILART          AS tipo_atividade_manutencao,

    regexp_replace(afih.QMNUM, '^0+', '')          
                        AS nota,
    afih.prioridade     AS texto_prioridade,
    afih.EQUNR          AS equipamento,
    afih.ADPSP          AS elem_ref_PM_PS,
    afih.ADDAT          AS data_referencia,
    aufk.ERDAT          AS data_entrada,

    regexp_replace(afih.WAPOS, '^0+', '')          
                        AS item_manutencao,
    afvc.LTXA1          AS texto_breve_operacao,

    aufk.DI_OPERATION_TYPE          AS aufk_DI_OPERATION_TYPE,
    aufk.DI_SEQUENCE_NUMBER         AS aufk_DI_SEQUENCE_NUMBER,

    afih.DI_OPERATION_TYPE          AS afih_DI_OPERATION_TYPE,
    afih.DI_SEQUENCE_NUMBER         AS afih_DI_SEQUENCE_NUMBER,

    afih.t356_t_DI_OPERATION_TYPE   AS t356_t_DI_OPERATION_TYPE,
    afih.t356_t_DI_SEQUENCE_NUMBER  AS t356_t_DI_SEQUENCE_NUMBER,

    iloa.DI_OPERATION_TYPE          AS iloa_DI_OPERATION_TYPE,
    iloa.DI_SEQUENCE_NUMBER         AS iloa_DI_SEQUENCE_NUMBER,

    afko.DI_OPERATION_TYPE          AS afko_DI_OPERATION_TYPE,
    afko.DI_SEQUENCE_NUMBER         AS afko_DI_SEQUENCE_NUMBER,

    afvc.DI_OPERATION_TYPE          AS afvc_DI_OPERATION_TYPE,
    afvc.DI_SEQUENCE_NUMBER         AS afvc_DI_SEQUENCE_NUMBER

FROM ordens_filtradas AS aufk

LEFT JOIN status_usuario AS su
    ON su.AUFNR = aufk.AUFNR

LEFT JOIN afih_filtrado AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN iloa_filtrado AS iloa
    ON iloa.MANDT = afih.MANDT AND iloa.ILOAN = afih.ILOAN

LEFT JOIN afko_filtrado AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN afvc_filtrado AS afvc
    ON afvc.MANDT = afko.MANDT AND afvc.AUFPL = afko.AUFPL

ORDER BY aufk.AUFNR